# Step 3: Baseline Modeling

This notebook trains a baseline Linear Regression model using Spark MLlib.
A Pandas + scikit-learn implementation is used only for validation and
comparison of results.


Start Spark & Load Features

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("RetailDemandBaseline") \
    .getOrCreate()

df_model = spark.read.parquet("../data/processed/daily_features")
df_model.show(5)


26/01/19 10:21:21 WARN Utils: Your hostname, MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 10.0.0.22 instead (on interface en0)
26/01/19 10:21:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 10:21:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/19 10:21:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/19 10:21:22 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


+----------+--------------+-----------+------------+-----+----+-----+-----+------------------+------------------+
|      date|daily_quantity|day_of_week|week_of_year|month|year|lag_1|lag_7|         rolling_7|        rolling_14|
+----------+--------------+-----------+------------+-----+----+-----+-----+------------------+------------------+
|2010-12-09|         19930|          5|          49|   12|2010|23117|26919| 23004.14285714286| 23004.14285714286|
|2010-12-10|         21097|          6|          49|   12|2010|19930|31329|22005.714285714286|         22619.875|
|2010-12-12|         10603|          1|          49|   12|2010|21097|16199|           20544.0|22450.666666666668|
|2010-12-13|         17727|          2|          50|   12|2010|10603|16450|19744.571428571428|           21265.9|
|2010-12-14|         20284|          3|          50|   12|2010|17727|21795|           19927.0| 20944.18181818182|
+----------+--------------+-----------+------------+-----+----+-----+-----+-------------

Verify Required Columns

In [2]:
df_model.printSchema()


root
 |-- date: date (nullable = true)
 |-- daily_quantity: long (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- week_of_year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- lag_1: long (nullable = true)
 |-- lag_7: long (nullable = true)
 |-- rolling_7: double (nullable = true)
 |-- rolling_14: double (nullable = true)



Assemble Feature Vector (Spark)

In [3]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

assembler = VectorAssembler(
    inputCols=[
        "lag_1",
        "lag_7",
        "rolling_7",
        "rolling_14",
        "day_of_week",
        "week_of_year",
        "month"
    ],
    outputCol="features"
)

df_final = assembler.transform(df_model) \
    .select(
        "features",
        col("daily_quantity").cast("double").alias("label")
    ) \
    .dropna()

df_final.show(5)


+--------------------+-------+
|            features|  label|
+--------------------+-------+
|[23117.0,26919.0,...|19930.0|
|[19930.0,31329.0,...|21097.0|
|[21097.0,16199.0,...|10603.0|
|[10603.0,16450.0,...|17727.0|
|[17727.0,21795.0,...|20284.0|
+--------------------+-------+
only showing top 5 rows



Train / Test Split

In [4]:
train_df, test_df = df_final.randomSplit([0.8, 0.2], seed=42)

print("Train rows:", train_df.count())
print("Test rows:", test_df.count())


Train rows: 252
Test rows: 46


Spark Linear Regression

In [5]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    featuresCol="features",
    labelCol="label"
)

lr_model = lr.fit(train_df)


26/01/19 10:23:32 WARN Instrumentation: [d0bfb61c] regParam is zero, which might cause numerical instability and overfitting.
26/01/19 10:23:32 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
26/01/19 10:23:32 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
26/01/19 10:23:32 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


Spark Predictions

In [6]:
spark_predictions = lr_model.transform(test_df)
spark_predictions.select("label", "prediction").show(5)


+-------+------------------+
|  label|        prediction|
+-------+------------------+
|13595.0| 7622.703374810619|
|13415.0|  9357.72369280981|
|14940.0|19085.187092851742|
|12263.0| 9209.583990084882|
|21589.0|10814.340963570725|
+-------+------------------+
only showing top 5 rows



Spark Evaluation

In [7]:
from pyspark.ml.evaluation import RegressionEvaluator

rmse_eval = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="rmse"
)

mae_eval = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="mae"
)

rmse_spark = rmse_eval.evaluate(spark_predictions)
mae_spark = mae_eval.evaluate(spark_predictions)

rmse_spark, mae_spark


(13191.143238433897, 7478.156513218914)

Convert Spark → Pandas

In [8]:
pdf = df_model.orderBy("date").toPandas()
pdf.head()


,date,daily_quantity,day_of_week,week_of_year,month,year,lag_1,lag_7,rolling_7,rolling_14
0,2010-12-09,19930,5,49,12,2010,23117,26919,23004.142857,23004.142857
1,2010-12-10,21097,6,49,12,2010,19930,31329,22005.714286,22619.875000
2,2010-12-12,10603,1,49,12,2010,21097,16199,20544.000000,22450.666667
3,2010-12-13,17727,2,50,12,2010,10603,16450,19744.571429,21265.900000
4,2010-12-14,20284,3,50,12,2010,17727,21795,19927.000000,20944.181818


Prepare Features & Target

In [9]:
feature_cols = [
    "lag_1",
    "lag_7",
    "rolling_7",
    "rolling_14",
    "day_of_week",
    "week_of_year",
    "month"
]

X = pdf[feature_cols]
y = pdf["daily_quantity"]


Time-aware Train/Test Split

In [10]:
split_index = int(len(pdf) * 0.8)

X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]


scikit-learn Linear Regression

In [11]:
from sklearn.linear_model import LinearRegression

sk_model = LinearRegression()
sk_model.fit(X_train, y_train)

y_pred_sk = sk_model.predict(X_test)


scikit-learn Evaluation

In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

rmse_sk = np.sqrt(mean_squared_error(y_test, y_pred_sk))
mae_sk = mean_absolute_error(y_test, y_pred_sk)

rmse_sk, mae_sk


(np.float64(14999.159790175487), 10870.668168591283)

Comparison Table

In [13]:
import pandas as pd

comparison = pd.DataFrame({
    "Model": ["Spark Linear Regression", "scikit-learn Linear Regression"],
    "RMSE": [rmse_spark, rmse_sk],
    "MAE": [mae_spark, mae_sk]
})

comparison


,Model,RMSE,MAE
0,Spark Linear Regression,13191.143238,7478.156513
1,scikit-learn Linear Regression,14999.159790,10870.668169


### Comparison Summary

Both Spark MLlib and scikit-learn Linear Regression models show comparable
performance. Minor differences are expected due to distributed execution
(Spark) versus in-memory computation (scikit-learn).

### Interpretation

Spark Linear Regression performed better due to distributed optimization
and consistent feature vectorization. The scikit-learn model is included
as a validation baseline to confirm trend consistency.
